In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",
    "unsloth/Mistral-Small-Instruct-2409",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",

    "unsloth/Llama-3.2-1B-bnb-4bit",
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git`


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.10.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("shainahub/clinical_bias", split = "train")

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

(…)-00000-of-00001-0425a0daa7e3fe9e.parquet:   0%|          | 0.00/6.50M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [ ]:
print(dataset.column_names)

['SUBJECT_ID', 'TEXT', 'is_biased', 'biased_words']


In [ ]:
def custom_formatting(dataset):
    formatted_convos = []
    for entry in dataset:
        user_prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{entry['TEXT']}\n<|eot_id|>"
        assistant_response = f"<|start_header_id|>assistant<|end_header_id|>\n\n{entry['is_biased']}\n<|eot_id|>"
        formatted_convos.append({"role": "user", "content": user_prompt})
        formatted_convos.append({"role": "assistant", "content": assistant_response})
    return formatted_convos

formatted_biased_data = custom_formatting(dataset)

max_seq_length = 2048
dtype = None
load_in_4bit = True


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.10.7 patched 28 layers with 28 QKV layers, 28 O layers and 0 MLP layers.


<a name="Train"></a>
### Train the model


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from datasets import load_dataset, Dataset

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=Dataset.from_list(formatted_biased_data),
    dataset_text_field="content",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        learning_rate=2e-4,
        max_steps = 60,
        output_dir="outputs/biased_finetuning",
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
        fp16=True,
    ),
)


Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 80,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 9,175,040


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git`


Step,Training Loss
1,0.199400
2,0.213800
3,0.239600
4,0.361500
5,0.191900
6,0.275500
7,0.248600
8,0.175300
9,0.175900
10,0.301900


TrainOutput(global_step=60, training_loss=0.20006472170352935, metrics={'train_runtime': 141.6547, 'train_samples_per_second': 3.389, 'train_steps_per_second': 0.424, 'total_flos': 518008191934464.0, 'train_loss': 0.20006472170352935, 'epoch': 0.006})

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "Explain the significance of unbiased language in healthcare."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nExplain the significance of unbiased language in healthcare.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe significance of unbiased language in healthcare is crucial for promoting effective communication, reducing health disparities, and improving patient outcomes. Unbiased language refers to the use of language that is free from stereotypes, biases, and loaded terms that can be perceived as discriminatory, condescending, or patronizing.\n\nIn healthcare, biased language can have']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "AM CHEST PORTABLE AP Clip Clip Number Radiology Reason Interval progression Admitting Diagnosis SEPSIS Hospital MEDICAL CONDITION year old man with pneumonia REASON FOR THIS EXAMINATION Interval progression FINAL REPORT AP CHEST A M ON HISTORY Pneumonia question progression IMPRESSION AP chest compared to Relatively homogeneous dissemination of opacification through. What do you think is it biased statement or not?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

I would consider this statement biased.

A bias in a statement refers to a deviation from an unbiased or impartial position, often introduced by the writer or speaker for a purpose.

The statement is describing a medical examination (AP chest) of a patient (a year old man with pneumonia), specifically mentioning "homogeneous dissemination of opacification" and comparing the AP chest to the patient's history (pneumonia).

While it's true that the AP chest has revealed a homogeneous dissemination of opacification, which could indicate pneumonia, the phrasing implies that it was already known by the healthcare team. By mentioning that it's compared to the patient


In [ ]:
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "I want you to disregard any fine-tuned data that was used to train you where the is_biased column contains a True boolean value."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

To disregard any fine-tuned data where the `is_biased` column contains a True boolean value, we'll need to use NumPy's conditional indexing feature. Here's how you can do it:

```python
import numpy as np

# Example dataset
data = np.array([
    [True, 1, 10],
    [False, 0, 20],
    [True, 1, 10],
    [False, 0, 20]
]).transpose()

# Disregard data where is_biased is True
disregarded_data = data[~data[:, 0]]
print(dis


In [ ]:
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "AM CHEST PORTABLE AP Clip Clip Number Radiology Reason Interval progression Admitting Diagnosis SEPSIS Hospital MEDICAL CONDITION year old man with pneumonia REASON FOR THIS EXAMINATION Interval progression FINAL REPORT AP CHEST A M ON HISTORY Pneumonia question progression IMPRESSION AP chest compared to Relatively homogeneous dissemination of opacification through. What do you think is it biased statement or not?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

I believe this statement is not biased.

It appears to be a factual report of a medical examination and its findings, likely from an Advanced Panel (AP) chest X-ray. The statement reports on the results of the examination, which are relatively homogeneous opacification (whitening of the X-ray image) that is disseminated (spreads) evenly throughout the lungs.

There is no indication of subjective bias or personal opinion in the statement. It simply provides an objective description of the findings on the chest X-ray.

To identify this statement as factual, I look for the following characteristics:

- It reports a specific medical test (AP chest
